In [ ]:
import pandas
import numpy
import torch

metallicity_table = {"[m/H]":1, "[M/H]":1, "[Fe/H]":2}
def metallicity_parser(input_string):
  return metallicity_table.get(input_string, numpy.nan)

def average_or_nan(series, default=numpy.nan):
  if (pandas.isna(series) != True).any():
    return series.mean()
  else:
    return default

def max_or_nan(series):
  if (pandas.isna(series) != True).any():
    return series.max()
  else:
    return numpy.nan

def find_metallicity_standard(series):
  uses_M_H = (series == 1)
  if uses_M_H.any():
    return 1, uses_M_H
  else:
    uses_Fe_H = (series == 2)
    if uses_Fe_H.any():
      return 2, uses_Fe_H
    else:
      return numpy.nan, uses_Fe_H

def parse_table(file_path, existing_stars, available_series):
  """
  Inputs:
    file_path: String.
    existing_stars: Set. Contains strings.
  Outputs:
    model_x1: 3D PyTorch tensor.
    model_x1_mask: 2D PyTorch tensor.
    model_y: 3D PyTorch tensor.
  """

  triplet_data = {("sy_pm", "sy_pmerr1", "sy_pmerr2"), ("sy_dist", "sy_disterr1", "sy_disterr2")}
  quadruplet_data = {("st_mass", "st_masserr1", "st_masserr2", "st_masslim"), ("st_met", "st_meterr1", "st_meterr2", "st_metlim"),
                     ("st_age", "st_ageerr1", "st_ageerr2", "st_agelim"), ("st_rotp", "st_rotperr1", "st_rotperr2", "st_rotplim")}

  imputable_data_planet_low = ["pl_orbpererr1", "pl_orbsmaxerr1", "pl_masseerr1", "pl_msinieerr1", "pl_orbeccenerr1"]
  imputable_data_planet_high = ["pl_orbpererr2", "pl_orbsmaxerr2", "pl_masseerr2", "pl_msinieerr2", "pl_orbeccenerr2"]
  imputable_data_planet_max = ["cb_flag"]
  planet_data_columns = imputable_data_planet_max + imputable_data_planet_low + imputable_data_planet_high

  imputable_data_star = ["st_masserr1", "st_masserr2", "st_ageerr1", "st_ageerr2", "st_rotperr1", "st_rotperr2",
                         "glat", "glon", "sy_pmerr1", "sy_pmerr2", "sy_disterr1", "sy_disterr2"]
  imputable_data_star_max = ["sy_snum"]
  other_data_star = ["st_metratio", "st_meterr1", "st_meterr2"]
  star_data_columns = imputable_data_star_max + other_data_star + imputable_data_star

  max_imputable_data = imputable_data_planet_max + imputable_data_star_max

  data_table = pandas.read_csv(file_path, converters={"st_metratio":metallicity_parser}, comment='#')
  data_table = data_table.loc[(data_table["hostname"] in existing_stars) and (data_table["rv_flag"] == 1) and (data_table["st_nrvc"] > 0)]

  model_x1_list = list()
  model_x1_paddings = list()
  model_y_list = list()

  max_planets_number = 0

  for star in existing_stars:
    planets_table = pandas.DataFrame(columns=data_table.columns)

    exoplanets = set(data_table.loc[data_table["hostname"] == star, "pl_name"])
    for exoplanet in exoplanets:
      use_sources = data_table.loc[data_table["pl_name"] == exoplanet]

      is_non_controversial = (use_sources["pl_controv_flag"] == 0)
      if is_non_controversial.any():
        use_sources = use_sources.loc[is_non_controversial]

      is_confirmed = (use_sources["soltype"] == "Published Confirmed")
      if is_confirmed.any():
        use_sources = use_sources.loc[is_confirmed]

      for row in use_sources.index:
        for (number, uncertainty_up, uncertainty_down) in triplet_data:
          if pandas.isna(use_sources.at[row, number]):
            use_sources.at[row, uncertainty_up] = numpy.nan
            use_sources.at[row, uncertainty_down] = numpy.nan
          else:
            if pandas.isna(use_sources.at[row, uncertainty_up]):
              use_sources.at[row, uncertainty_up] = 0
            use_sources.at[row, uncertainty_up] += use_sources.at[row, number]
            if pandas.isna(use_sources.at[row, uncertainty_down]):
              use_sources.at[row, uncertainty_down] = 0
            use_sources.at[row, uncertainty_down] += use_sources.at[row, number]

        for (number, uncertainty_up, uncertainty_down, limit) in quadruplet_data:
          if pandas.isna(use_sources.at[row, number]):
            use_sources.at[row, uncertainty_up] = numpy.nan
            use_sources.at[row, uncertainty_down] = numpy.nan
          else:
            if pandas.isna(use_sources.at[row, uncertainty_up]):
              use_sources.at[row, uncertainty_up] = 0
            use_sources.at[row, uncertainty_up] += use_sources.at[row, number]
            if(use_sources.at[row, limit] == 1):
              use_sources.at[row, uncertainty_down] = numpy.nan
            else:
              if pandas.isna(use_sources.at[row, uncertainty_down]):
                use_sources.at[row, uncertainty_down] = 0
              use_sources.at[row, uncertainty_down] += use_sources.at[row, number]

      is_default_data = (use_sources["default_flag"] == 1)
      if is_default_data.any():
        planets_table.loc[exoplanet] = use_sources.iloc[is_default_data.argmax()]
      else:
        planets_table.loc[exoplanet] = None

      for column_label in imputable_data_planet_low:
        if pandas.isna(planets_table.at[exoplanet, column_label]):
          planets_table.at[exoplanet, column_label] = average_or_nan(use_sources[column_label], default=0)
      for column_label in imputable_data_planet_high:
        upper_bound = float('inf')
        if(column_label == "pl_orbeccenerr2"):
          upper_bound = 1
        if pandas.isna(planets_table.at[exoplanet, column_label]):
          planets_table.at[exoplanet, column_label] = average_or_nan(use_sources[column_label], default=upper_bound)
      for column_label in imputable_data_star:
        if pandas.isna(planets_table.at[exoplanet, column_label]):
          planets_table.at[exoplanet, column_label] = average_or_nan(use_sources[column_label], default=numpy.nan)

      for column_label in max_imputable_data:
        if pandas.isna(planets_table.at[exoplanet, column_label]):
          planets_table.at[exoplanet, column_label] = max_or_nan(use_sources[column_label])

      if pandas.isna(planets_table.at[exoplanet, "st_metratio"]):
        planets_table.at[exoplanet, "st_metratio"], data_locations = find_metallicity_standard(use_sources["st_metratio"])
        planets_table.at[exoplanet, "st_meterr1"] = average_or_nan(use_sources.loc[data_locations, "st_meterr1"], default=numpy.nan)
        planets_table.at[exoplanet, "st_meterr2"] = average_or_nan(use_sources.loc[data_locations, "st_meterr2"], default=numpy.nan)
      else:
        same_unit = (use_sources["st_metratio"] == planets_table.at[exoplanet, "st_metratio"])
        if pandas.isna(planets_table.at[exoplanet, "st_meterr1"]):
          planets_table.at[exoplanet, "st_meterr1"] = average_or_nan(use_sources.loc[same_unit, "st_meterr1"], default=numpy.nan)
        if pandas.isna(planets_table.at[exoplanet, "st_meterr2"]):
          planets_table.at[exoplanet, "st_meterr2"] = average_or_nan(use_sources.loc[same_unit, "st_meterr2"], default=numpy.nan)

    planets_data = (planets_table[planet_data_columns]).sort_values(planet_data_columns)
    model_y_list.append([(planets_data.loc[row_name]).to_numpy(dtype=numpy.double, copy=True) for row_name in planets_table.index])
    max_planets_number = max(max_planets_number, len(model_y_list[-1]))

    star_data = dict.fromkeys(star_data_columns, numpy.nan)
    for column_label in imputable_data_star:
      star_data[column_label] = average_or_nan(planets_table[column_label], default=numpy.nan)
    for column_label in imputable_data_star_max:
      star_data[column_label] = max_or_nan(planets_table[column_label])
    star_data["st_metratio"], data_locations = find_metallicity_standard(planets_table["st_metratio"])
    star_data["st_meterr1"] = average_or_nan(planets_table.loc[data_locations, "st_meterr1"], default=numpy.nan)
    star_data["st_meterr2"] = average_or_nan(planets_table.loc[data_locations, "st_meterr2"], default=numpy.nan)

    star_data_list = list()
    padding_list = (available_series[star]).copy()
    for encoding_position, column_name in enumerate(star_data_columns):
      encoding_vector = numpy.zeros(len(star_data_columns), dtype=numpy.double)
      if not pandas.isna(star_data[column_name]):
        encoding_vector[encoding_position] = star_data[column_name]
        padding_list.append(False)
      else:
        padding_list.append(True)
      star_data_list.append(encoding_vector)
    model_x1_list.append(star_data_list)
    model_x1_paddings.append(padding_list)

  for y_entry in model_y_list:
    for padding in range(max_planets_number-len(y_entry)):
      y_entry.append(numpy.zeros(len(planet_data_columns), dtype=numpy.double))

  model_x1 = torch.from_numpy(numpy.asarray(model_x1_list, dtype=numpy.double))
  model_x1_mask = torch.from_numpy(numpy.asarray(model_x1_paddings, dtype=numpy.bool_))
  model_y = torch.from_numpy(numpy.asarray(model_y_list, dtype=numpy.double))

  return model_x1, model_x1_mask, model_y